In [1]:
!pip install git+https://github.com/TracingThePast/automatic_matching

  Cloning https://github.com/TracingThePast/automatic_matching to /private/var/folders/99/chbgg_5s3tn77pkyp7wp3_bm0000gn/T/pip-req-build-8f4083in
  Running command git clone --filter=blob:none --quiet https://github.com/TracingThePast/automatic_matching /private/var/folders/99/chbgg_5s3tn77pkyp7wp3_bm0000gn/T/pip-req-build-8f4083in
  Resolved https://github.com/TracingThePast/automatic_matching to commit 12818379a4c891d743a0df29f95fa2058d729ef3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
from IPython.display import HTML, display
from automatic_matching import get_matching_score, get_results_as_html

## Ablauf:

Die Felder `forenames`, `surnames`, `birth_place` und `death_place` werden soweit es geht in ihre einzelnen Bestandteile zerlegt, das heißt die Aneinanderreihung von Namen `Anna, Anne Anneliese` wird zerlegt in `['Anna', 'Anne', 'Anneliese']` damit die einzelnen Namen jeweils direkt verglichen werden können. Beim Vergleichen wird dann immer das Wertepaar ausgewählt, welches die geringste Levenshtein Distanz aufweist. Abschließend wird aus dem Satz der Minimaldistanzen aller möglichen Paare der Mittelwert berechnet. 

In [4]:


local_test_data = {
    'forenames': 'Anna, Anne Anneliese',
    'surnames': 'Musterfrau',
    'birth_place': 'München',
    'birth_date': '02.03.1923',
    'death_place': 'Dachau',
    'death_date': '11.12.1941',
}

external_test_data = {
    'forenames': 'Anne Annelies',
    'surnames': 'Musterphrau',
    'birth_place': 'München / Bavaria',
    'birth_date': '12.12.1923',
#    'death_place': 'Dachau',
#    'death_date': '11.11.1941',
}

result = get_matching_score(local_test_data, external_test_data)
output_html = get_results_as_html(result)

display(HTML(output_html))

Local value,External value,Levenshtein distance
"{'original': 'Anne', 'normalized': 'ane'}","{'original': 'Anne', 'normalized': 'ane'}",0 (0)
"{'original': 'Anna', 'normalized': 'ana'}","{'original': 'Anne', 'normalized': 'ane'}",1 (1)
"{'original': 'Anneliese', 'normalized': 'aneliese'}","{'original': 'Annelies', 'normalized': 'anelies'}",1 (1)
"{'original': 'Anne', 'normalized': 'ane'}","{'original': 'Annelies', 'normalized': 'anelies'}",4 (4)
"{'original': 'Anna', 'normalized': 'ana'}","{'original': 'Annelies', 'normalized': 'anelies'}",5 (5)
Local value,External value,Levenshtein distance
"{'original': 'Musterfrau', 'normalized': 'musterfrau'}","{'original': 'Musterphrau', 'normalized': 'musterfrau'}",0 (2)
Local value,External value,Levenshtein distance
02.03.1923,12.12.1923,3
Local value,External value,Levenshtein distance
